In [1]:
import pandas as pd
import numpy as np
import time

In [2]:
start_time = time.time()

In [3]:
df = pd.read_csv("../examples/products.csv")

In [4]:
#df = df.head(150)

In [5]:
df["name"].str.strip().str.split(" ")

0        [Home, komposztálható, bevásárlószatyor, 209x2...
1                     [Auchan, bevásárlótáska,, duplafülű]
2              [Zöldség-gyümölcs, retasak, 3db, 28x34, cm]
3              [Auchan, karton, doboz, 39,8x29,8x30,4, cm]
4        [Auchan, ünnepi, bevásárló, doboz, 39,8x29,8x3...
                               ...                        
15210    [Gierlinger's, főtt,, füstölt, sertés, backbac...
15211    [Auchan, Nívó, Vöröshagymás, sertés, tepertőkr...
15212    [Auchan, Nívó, snidlinges, májas, nem, ehető, ...
15213     [Kometa, Útravaló, étkezesi, sertészsír, 400, g]
15214                                  [Kacsazsír, 200, g]
Name: name, Length: 15215, dtype: object

In [6]:
name_split = df["name"].str.strip().str.replace("  ", " ").str.split(" ")

In [7]:
df["unit"] = name_split.str[-1].str.lower().str.strip()
df["unit size"] = None
df[["unit", "unit size"]]

,unit,unit size
0,cm,None
1,duplafülű,None
2,cm,None
3,cm,None
4,cm,None
...,...,...
15210,g,None
15211,g,None
15212,g,None
15213,g,None


In [8]:
def check_numeric(x):
    try:
        float(x)
        return True
    except:
        return False

In [9]:
def clean_unit(unit):
    g_mask = df["unit"].str.contains(unit, regex=True, na=False)
    extracted_values = df.loc[g_mask, "unit"].str.split(unit).str[0].str.replace(",", ".")
    extracted_values_mask = extracted_values.apply(check_numeric)
    extracted_values.loc[~extracted_values_mask] = None
    df.loc[g_mask, "unit size"] = extracted_values
    df.loc[g_mask, "unit"] = df.loc[g_mask, "unit"].where(~extracted_values_mask, other=unit)

In [10]:
for unit in ["db", "cm", "g", "kg", "l", "m", "ml", "gr"]:
    clean_unit(unit)

In [11]:
#g_mask = df["unit"].str.contains("g", regex=True, na=False)

In [12]:
#extracted_values = df.loc[g_mask, "unit"].str.split("g").str[0].str.replace(",", ".")

In [13]:
#extracted_values_mask = extracted_values.apply(check_numeric)

In [14]:
#extracted_values.loc[~extracted_values_mask] = None

In [15]:
#df.loc[g_mask, "unit size"] = extracted_values
#df.loc[g_mask, "unit"] = df.loc[g_mask, "unit"].where(~extracted_values_mask, other="g")

In [16]:
df[["unit size", "unit"]]

,unit size,unit
0,None,cm
1,None,duplafülű
2,None,cm
3,None,cm
4,None,cm
...,...,...
15210,None,g
15211,None,g
15212,None,g
15213,None,g


In [17]:
df["unit"] = df["unit"].replace("\)", "", regex=True)

In [18]:
not_unit = ~df["unit"].isin(["db", "cm", "g", "kg", "l", "ml", "m", "gr"])

In [19]:
df.loc[not_unit, "unit"] = None

In [20]:
df["unit"].isnull().sum()

1549

In [21]:
unit_size = name_split.str[-2].str.replace("(", "", regex=True).str.replace(",", ".").str.strip()

In [22]:
set_unit_size_mask = ~df["unit size"].isnull()

In [23]:
df.loc[~set_unit_size_mask, "unit size"] = unit_size

In [24]:
numeric_kind = df["unit size"].apply(check_numeric)

In [25]:
area_kind = df["unit size"].str.find("x") != -1

In [26]:
keep = (numeric_kind | area_kind) & ~df["unit"].isnull()

In [27]:
df.loc[~keep, "unit size"] = None

In [28]:
df = df.reindex(columns=["name", "price", "unit size", "unit", "id", "defaultVariant", "productId", "categoryId"])
df.loc[~df["unit"].isnull(), ["name", "unit size", "unit"]]

,name,unit size,unit
0,Home komposztálható bevásárlószatyor 209x2x75x...,209x2x75x600,cm
2,Zöldség-gyümölcs retasak 3db 28x34 cm,28x34,cm
3,"Auchan karton doboz 39,8x29,8x30,4 cm",39.8x29.8x30.4,cm
4,"Auchan ünnepi bevásárló doboz 39,8x29,8x30,4 cm",39.8x29.8x30.4,cm
5,"Kelly uzsonnás és hűtőtasak 2 l, 100 db",100,db
...,...,...,...
15210,"Gierlinger's főtt, füstölt sertés backbacon 200 g",200,g
15211,Auchan Nívó Vöröshagymás sertés tepertőkrém ro...,250,g
15212,Auchan Nívó snidlinges májas nem ehető műbélbe...,400,g
15213,Kometa Útravaló étkezesi sertészsír 400 g,400,g


### TODO remove unit from name

In [29]:
df["unit"].isnull().sum()

1549

In [30]:
len(df)-df["unit"].isnull().sum()

13666

In [31]:
df.loc[df["unit size"].isnull() & ~df["unit"].isnull()]

,name,price,unit size,unit,id,defaultVariant,productId,categoryId
264,Auchan Nívó Gyorsfagyasztott tenger gyümölcsei...,1500,None,g,524575,4061965,524575,5921
275,Garnélafarok előfőz 500/400 g,4879,None,g,34183,34186,34183,5916
660,Carte D'Or poharas jégkrém vegán Kakaós-Pirosg...,2049,None,ml,410923,3948289,410923,5936
880,Lazacfilé bőr nélkül 540/600 g,9449,None,g,533729,4071119,533729,5917
1787,CEBU ÖKO Húzófüles szemeteszsák 35L /20 DB,629,None,db,513959,4051349,513959,5943
...,...,...,...,...,...,...,...,...
14406,"1M Silly Val..DD 8X0,33 l -1X0,25 l",6799,None,l,578703,4116093,578703,6124
14720,Syoss Root Retoucher lenövést elfedő hajszínez...,2599,None,ml,68090,3605171,68090,6208
14910,DARNÓ SZAFALÁDÉ 2X150 g,1149,None,g,469214,4006586,469214,6001
15175,Paté du Chef pástétomválogatás 6*25 g,1299,None,g,366899,3904181,366899,6002


In [32]:
time.time()-start_time

0.8181796073913574

In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15215 entries, 0 to 15214
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   name            15215 non-null  object
 1   price           15215 non-null  int64 
 2   unit size       13566 non-null  object
 3   unit            13666 non-null  object
 4   id              15215 non-null  int64 
 5   defaultVariant  15215 non-null  int64 
 6   productId       15215 non-null  int64 
 7   categoryId      15215 non-null  int64 
dtypes: int64(5), object(3)
memory usage: 951.1+ KB
